In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.datasets import Food101
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import time

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets
train_dataset = Food101(root='./data', split='train', transform=transform, download=True)
test_dataset = Food101(root='./data', split='test', transform=transform, download=True)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# Evaluate the model on the test set
def evaluate_model(model, test_loader, device):
    """
    Evaluate the model on the test set.

    Args:
    - model: PyTorch model to evaluate
    - test_loader: DataLoader for the test dataset
    - device: device to run the evaluation on (e.g., 'cuda' or 'cpu')

    Returns:
    - all_predictions: List of predicted labels
    - all_labels: List of ground truth labels
    """
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_predictions

# Define custom classifier
class CustomClassifier(nn.Module):
    def __init__(self, num_classes):
        super(CustomClassifier, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(1280, 256)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout3(x)
        x = self.fc3(x)
        return x

num_classes = len(train_dataset.classes)  
pretrained = models.mobilenet_v2(pretrained=True)
pretrained = nn.Sequential(*list(pretrained.children())[:-1])  # Exclude the last FC layer

# Freeze pre-trained weights
for param in pretrained.parameters():
    param.requires_grad = False

# Define the model
model = nn.Sequential(
    pretrained,
    CustomClassifier(num_classes)
)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

start = time.time()

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')
model.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Calculate accuracy

    all_labels, all_predictions = evaluate_model(model, test_loader, device)
    accuracy = accuracy_score(all_labels, all_predictions)
    print(f"Test Accuracy: {accuracy:.4f} at Epoch [{epoch+1}/{num_epochs}]")

print(f'Training duration: {time.time()-start}')

torch.save(model.state_dict(), 'mobilenet_model_1.pkl')


c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Device: cuda
Epoch [1/10], Loss: 2.5825
Test Accuracy: 0.5428 at Epoch [1/10]
Epoch [2/10], Loss: 2.1656
Test Accuracy: 0.5638 at Epoch [2/10]
Epoch [3/10], Loss: 2.0816
Test Accuracy: 0.5783 at Epoch [3/10]
Epoch [4/10], Loss: 2.0355
Test Accuracy: 0.5850 at Epoch [4/10]
Epoch [5/10], Loss: 1.9953
Test Accuracy: 0.5865 at Epoch [5/10]
Epoch [6/10], Loss: 1.9779
Test Accuracy: 0.5885 at Epoch [6/10]
Epoch [7/10], Loss: 1.9469
Test Accuracy: 0.5925 at Epoch [7/10]
Epoch [8/10], Loss: 1.9302
Test Accuracy: 0.5941 at Epoch [8/10]
Epoch [9/10], Loss: 1.9137
Test Accuracy: 0.5983 at Epoch [9/10]
Epoch [10/10], Loss: 1.8933
Test Accuracy: 0.6014 at Epoch [10/10]
Training duration: 7216.362441301346
